In [1]:
!pip install --user --upgrade pip
!pip install --user --upgrade kfp

  Using cached pip-21.0.1-py3-none-any.whl (1.5 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 159 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 8.7 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 2.1 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 8.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 10.0 MB/s eta 0:00:01
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=222156 sha256=6e57f459c4582d95d0bb96f286d54bec2339b8917e342eb1caf057cb3334bc9e
  Stored in directory: /home/jovyan/.cache/pip/wheels/88/63/63/f727a62aaba1e0fe13fe549e1b7538e9b8a2bc43dcae8138c8
  Created wheel for docstring-parser: 

  Created wheel for strip-hints: filename=strip_hints-0.1.9-py2.py3-none-any.whl size=24671 sha256=adf1fd22b46bba60810fa5c6ce90163692160824764f33a67130deddb40e948a
  Stored in directory: /home/jovyan/.cache/pip/wheels/21/6d/fa/7ed7c0560e1ef39ebabd5cc0241e7fca711660bae1ad752e2b
Successfully built kfp docstring-parser fire kfp-server-api strip-hints
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Hyperparameters
Registry = 'ckwlsgur20'
ImageNAME = 'mnist'
ImageVer = '4.0v'

In [5]:
def print_op(msg):
    return dsl.ContainerOp(
        name='Print',
        image='alpine:3.6',
        command=['echo', msg],
    )

In [30]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler


@dsl.pipeline(
    name="mnist_pipeline",
    description="mnist_pipeline with pv and pvc"
)
def mnist_pipeline(epochs: int):
    train_test_step = dsl.ContainerOp(
        name="train and test",
        image="{}/{}:{}".format(Registry, ImageNAME, ImageVer),
        arguments=[
            epochs
        ],
        file_outputs={
            'accuracy': '/accuracy.json',
            'mlpipeline-metrics': '/mlpipeline-metrics.json'
        },
        pvolumes={
            "/mnt": dsl.PipelineVolume(pvc="kubeflow")
        },
    )

    baseline = 0.99
    with dsl.Condition(train_test_step.outputs['accuracy'] > baseline):
        print_op('accuracy is higher than {}'.format(baseline))
    with dsl.Condition(train_test_step.outputs['accuracy'] <= baseline):
        print_op('accuracy is lower than {}'.format(baseline))

In [31]:
# Hyperparameters
pipeline_func = mnist_pipeline
ExpName = '4.pytorch-mnist'
Args = {"epochs": 10}
arguments = {"data_path":DATA_PATH,
             "model_file":MODEL_PATH,
             "image_number": IMAGE_NUMBER}

NameError: name 'DATA_PATH' is not defined

In [34]:
# compile pipeline
from datetime import datetime
runtime = '{}'.format(datetime.now().strftime("%Y%m%d %H%M%S"))
run_name = pipeline_func.__name__ + ' run ' + runtime
kfp.compiler.Compiler().compile(pipeline_func, '{}.zip'.format(ExpName))
# Submit pipeline to kubeflow server directly
client = kfp.Client(host='pipelines-api.kubeflow.svc.cluster.local:8888')
run_result = client.create_run_from_pipeline_func(pipeline_func,
                                                  experiment_name=ExpName,
                                                  run_name=run_name,
                                                  arguments=Args)